In [1]:
import transformers 
import datasets
import torch
import logging

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# squad = datasets.load_dataset("squad",split="train[:6000]")
squad = datasets.load_dataset("squad")


Found cached dataset squad (C:/Users/dama_/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 34.01it/s]


In [5]:
# torch.save(squad,"../datasets/squad.pt")

In [36]:
# squad = squad.train_test_split(test_size=0.2)

In [2]:
# squad=torch.load("../datasets/squad.pt")

In [3]:
squad['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [3]:
# Preprocessing
# sep_token = '<sep>'
dataset_name = "squad"
model_type="roberta"
model_name= "roberta-base"
models_dir = "saved_models/roberta-base_mod_squad"
checkpoint = 'roberta-base'
max_input_length = 386


# ## Training
learning_rate = 3e-5
num_epochs = 3

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [5]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_input_length ,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Loading cached processed dataset at C:\Users\dama_\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-0ecdefe9411ff510.arrow
Loading cached processed dataset at C:\Users\dama_\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-4c8f1006291bcc97.arrow


In [7]:
data_collator = transformers.DefaultDataCollator()

In [8]:
model = transformers.AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use 

In [9]:
training_args = transformers.TrainingArguments(
    output_dir=models_dir,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

e:\HOGENT\2022_2023\BA\BP_Info_Support\bert\saved_models/roberta-base_mod_squad is already a clone of https://huggingface.co/damapika/roberta-base_mod_squad. Make sure you pull the latest changes with `repo.git_pull()`.


In [10]:
torch.cuda.empty_cache()

In [23]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 4            |        cudaMalloc retries: 7         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   7249 MiB |   7283 MiB | 324836 MiB | 317587 MiB |\n|       from large pool |   7247 MiB |   7281 MiB | 324750 MiB | 317503 MiB |\n|       from small pool |      1 MiB |      3 MiB |     86 MiB |     84 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   7249 MiB |   7283 MiB | 324836 MiB | 317587 MiB |\n|       from large pool |   7247 MiB |   7281 MiB |

In [11]:
trainer.train()

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: damapika. Use `wandb login --relogin` to force relogin


  2%|▏         | 500/32850 [06:12<8:17:50,  1.08it/s] 

{'loss': 1.9319, 'learning_rate': 2.954337899543379e-05, 'epoch': 0.05}


  3%|▎         | 1000/32850 [11:17<5:11:04,  1.71it/s]

{'loss': 1.2724, 'learning_rate': 2.908675799086758e-05, 'epoch': 0.09}


...

 99%|█████████▉| 32500/32850 [14:06:06<10:03,  1.72s/it]  

{'loss': 0.4838, 'learning_rate': 3.19634703196347e-07, 'epoch': 2.97}


                                                        
100%|██████████| 32850/32850 [14:27:23<00:00,  1.58s/it]

{'eval_loss': 1.010042667388916, 'eval_runtime': 648.9805, 'eval_samples_per_second': 16.287, 'eval_steps_per_second': 2.037, 'epoch': 3.0}
{'train_runtime': 52046.3868, 'train_samples_per_second': 5.049, 'train_steps_per_second': 0.631, 'train_loss': 0.7432580640726075, 'epoch': 3.0}


TrainOutput(global_step=32850, training_loss=0.7432580640726075, metrics={'train_runtime': 52046.3868, 'train_samples_per_second': 5.049, 'train_steps_per_second': 0.631, 'train_loss': 0.7432580640726075, 'epoch': 3.0})

In [12]:
trainer.push_to_hub()

Upload file pytorch_model.bin: 100%|█████████▉| 473M/473M [00:45<00:00, 10.9MB/s] To https://huggingface.co/damapika/roberta-base_mod_squad
   42c76d4..cea1e0c  main -> main

Upload file pytorch_model.bin: 100%|██████████| 473M/473M [00:46<00:00, 10.8MB/s]
Upload file runs/May18_16-37-23_Damapika/events.out.tfevents.1684420652.Damapika.2380.0: 100%|██████████| 15.2k/15.2k [00:46<00:00, 338B/s]  
To https://huggingface.co/damapika/roberta-base_mod_squad
   cea1e0c..e9d04f5  main -> main



'https://huggingface.co/damapika/roberta-base_mod_squad/commit/cea1e0cabd7f54dba7e5dee294bd6e142310df72'

In [13]:
question = "Who is Fyodor Dostoevsky?"
context = "In the world of literature, there have been many authors who have gained a reputation for their ability to create complex characters. One such author is Fyodor Dostoevsky, a Russian novelist who wrote several influential works in the 19th century."

In [14]:
question_answerer = transformers.pipeline("question-answering", model="damapika/roberta-base_mod_squad")
question_answerer(question=question, context=context)

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dama_\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


{'score': 0.88525390625,
 'start': 172,
 'end': 190,
 'answer': 'a Russian novelist'}